## Import 1: Classes & Path

In [34]:
# Import 1. 
# Import functions.py into notebook from src folder. 
import sys

# Path relative to your notebook.
sys.path.append("/Users/joel/Desktop/daimil10/CAPSTONE_2/Niger-Political-Shift-Examination/src")

# Import my .py classes.
import data_clean

# Auto reload.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import 2: All Libs

In [35]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style

## Read File & Inital Drop of Columns

In [36]:
# Read file in.
# Dropping columns to trim data to 1990-2022.
# Use clean class and init a cleaner object.
cleaner = data_clean.Clean()

niger = cleaner.read_file('/Users/joel/Desktop/daimil10/CAPSTONE_2/Niger-Political-Shift-Examination/data/Niger.csv')
niger = niger.drop(['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989'], axis=1)

# Make a metadata df for indicator name, source note, and source_org just in case.
niger_meta = cleaner.read_file('/Users/joel/Desktop/daimil10/CAPSTONE_2/Niger-Political-Shift-Examination/data/Metadata_Indicator_Descriptors.csv')

niger.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1990,1991,1992,1993,1994,1995,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Niger,NER,Consumption of iodized salt (% of households),SN.ITK.SALT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Niger,NER,"International migrant stock, total",SM.POP.TOTL,"115,464",NaN,NaN,NaN,NaN,"145,999",...,NaN,NaN,"189,255",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Niger,NER,"Children in employment, wage workers (% of chi...",SL.WAG.0714.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Niger,NER,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN
4,Niger,NER,"Unemployment, female (% of female labor force)...",SL.UEM.TOTL.FE.NE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN


## Describing the Dataset

In [37]:
# Describing the Dataset 1.
niger.describe()

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
count,632,674,715,635,631,672,722,701,754,713,...,959,"1,061",991,973,"1,016",954,896,793,665,288
mean,"44,754,487,787","41,421,897,347","37,535,859,500","42,621,789,647","48,140,512,112","47,630,554,221","45,494,970,418","48,817,802,216","50,890,872,063","53,374,922,619",...,"110,059,991,606","106,360,325,558","119,756,320,971","127,710,184,521","129,087,917,014","149,650,645,622","169,730,885,040","185,340,190,226","244,117,752,625","481,601,657,597"
std,"247,665,773,031","240,071,715,589","228,423,822,206","247,164,556,463","263,691,546,529","267,810,497,969","259,405,937,215","273,512,924,129","292,020,157,481","301,238,051,369",...,"644,964,890,427","653,849,628,223","720,788,282,072","768,337,306,071","796,619,602,182","894,314,837,433","978,037,964,834","1,039,726,850,972","1,220,390,241,563","1,804,194,239,125"
min,"-52,400,000,000","-58,400,000,000","-46,100,000,000","-60,180,945,700","-79,200,000,000","-65,527,000,000","-73,200,000,000","-81,525,000,000","-110,403,000,000","-94,715,000,000",...,"-623,032,000,000","-742,860,000,000","-926,952,000,000","-757,983,000,000","-842,397,000,000","-1,029,010,000,000","-1,125,410,000,000","-1,225,480,000,000","-1,401,350,000,000","-1,625,610,000,000"
25%,7,5,5,5,6,5,5,6,6,6,...,6,6,5,5,7,5,6,4,6,4
50%,70,52,45,54,68,66,48,62,48,58,...,45,40,42,42,37,40,45,42,55,34
75%,"6,932,758","2,516,137","745,000","5,316,327","6,173,293","4,609,173","2,233,919","2,677,496","1,444,991","2,450,000",...,"45,100","1,170","10,088","23,837","7,814","31,304","321,889","703,879","6,799,690","6,347,849"
max,"2,450,170,000,000","2,482,210,000,000","2,356,980,000,000","2,455,630,000,000","2,597,110,000,000","2,708,270,000,000","2,650,270,000,000","2,715,640,000,000","2,938,890,000,000","2,921,010,000,000",...,"5,829,080,000,000","6,252,250,000,000","6,652,170,000,000","6,909,400,000,000","7,337,100,000,000","8,143,540,000,000","8,693,310,000,000","9,136,430,000,000","9,672,180,000,000","10,339,300,000,000"


In [38]:
# Describing the Dataset 2.
niger.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1478 entries, 0 to 1477
Data columns (total 37 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    1478 non-null   object 
 1   Country Code    1478 non-null   object 
 2   Indicator Name  1478 non-null   object 
 3   Indicator Code  1478 non-null   object 
 4   1990            632 non-null    float64
 5   1991            674 non-null    float64
 6   1992            715 non-null    float64
 7   1993            635 non-null    float64
 8   1994            631 non-null    float64
 9   1995            672 non-null    float64
 10  1996            722 non-null    float64
 11  1997            701 non-null    float64
 12  1998            754 non-null    float64
 13  1999            713 non-null    float64
 14  2000            884 non-null    float64
 15  2001            854 non-null    float64
 16  2002            867 non-null    float64
 17  2003            890 non-null    f

In [39]:
# Describing the Dataset 3.
niger.index

RangeIndex(start=0, stop=1478, step=1)

In [40]:
# Describing the Dataset 4.
niger.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021', '2022'],
      dtype='object')

In [41]:
# Describing the Dataset 5.
missing_v = niger.isnull().sum()
missing_v

Country Name         0
Country Code         0
Indicator Name       0
Indicator Code       0
1990               846
1991               804
1992               763
1993               843
1994               847
1995               806
1996               756
1997               777
1998               724
1999               765
2000               594
2001               624
2002               611
2003               588
2004               576
2005               502
2006               462
2007               500
2008               516
2009               473
2010               472
2011               435
2012               400
2013               519
2014               417
2015               487
2016               505
2017               462
2018               524
2019               582
2020               685
2021               813
2022              1190
dtype: int64

In [42]:
# Describing the Dataset 6.
# Using the function that spits outdict of col names and dtypes. 
cleaner.get_column_features(niger)

{'Country Name': dtype('O'),
 'Country Code': dtype('O'),
 'Indicator Name': dtype('O'),
 'Indicator Code': dtype('O'),
 '1990': dtype('float64'),
 '1991': dtype('float64'),
 '1992': dtype('float64'),
 '1993': dtype('float64'),
 '1994': dtype('float64'),
 '1995': dtype('float64'),
 '1996': dtype('float64'),
 '1997': dtype('float64'),
 '1998': dtype('float64'),
 '1999': dtype('float64'),
 '2000': dtype('float64'),
 '2001': dtype('float64'),
 '2002': dtype('float64'),
 '2003': dtype('float64'),
 '2004': dtype('float64'),
 '2005': dtype('float64'),
 '2006': dtype('float64'),
 '2007': dtype('float64'),
 '2008': dtype('float64'),
 '2009': dtype('float64'),
 '2010': dtype('float64'),
 '2011': dtype('float64'),
 '2012': dtype('float64'),
 '2013': dtype('float64'),
 '2014': dtype('float64'),
 '2015': dtype('float64'),
 '2016': dtype('float64'),
 '2017': dtype('float64'),
 '2018': dtype('float64'),
 '2019': dtype('float64'),
 '2020': dtype('float64'),
 '2021': dtype('float64'),
 '2022': dtype('

## Project Goal #1:
Examine overall Political Stability Index from 1990 to 2022 (32 year span).

## Project Goal #2:
Explore Civil Unrest (Indicators and Incidents) over specified time frame.

## Project Goal #3:
Highlight Political Regime Metrics (multiple coups and multiple leaders). 

## Project Goal #4:
Uncover data points related to corruption perception index & economic indicators. 


## Project Goal #5:
Hypo test (1, 2, 3, 4, or 5)

## Project Goal #6:
Linear Model (1, 2, or 3)